In [1]:
import pandas as pd
import numpy as np
import math
import re
import datetime

In [2]:
hb_file_path = "./data/gwangjingu_hostess_bar.csv"
eb_file_path = "./data/gwangjingu_entertainment_bar.csv"

In [3]:
hb = pd.read_csv(hb_file_path,encoding="CP949")
print(hb.head(3))
print(hb.info)

   개방자치단체코드                    관리번호     인허가일자  인허가취소일자  영업상태코드  영업상태명  \
0   3040000  3040000-103-1993-06863  19931030      NaN       1  영업/정상   
1   3040000  3040000-103-1993-06865  19931122      NaN       3     폐업   
2   3040000  3040000-103-1993-06871  19931026      NaN       3     폐업   

   상세영업상태코드 상세영업상태명        폐업일자  휴업시작일자  ...  공장판매직종업원수  공장생산직종업원수 건물소유구분명  \
0         1      영업         NaN     NaN  ...        NaN        NaN     NaN   
1         2      폐업  20001226.0     NaN  ...        NaN        NaN     NaN   
2         2      폐업  19950403.0     NaN  ...        NaN        NaN     NaN   

   보증액 월세액 다중이용업소여부   시설총규모  전통업소지정번호 전통업소주된음식 홈페이지  
0  NaN NaN        Y   97.94       NaN      NaN  NaN  
1  NaN NaN        N  113.82       NaN      NaN  NaN  
2  NaN NaN        N   68.88       NaN      NaN  NaN  

[3 rows x 44 columns]
<bound method DataFrame.info of      개방자치단체코드                    관리번호     인허가일자  인허가취소일자  영업상태코드  영업상태명  \
0     3040000  3040000-103-1993-06863  19931030 

In [4]:
eb = pd.read_csv(eb_file_path,encoding="CP949")

In [5]:
len(hb.loc[hb["관리번호"]=='3040000-103-1993-06928',["좌표정보(X)","좌표정보(Y)"]]["좌표정보(X)"])
# 좌표정보가 없는 없체들은 NaN.

1

### filter_no_location은 좌표정보가 NaN인 row를 걸러냄

In [6]:
hb_filter_no_location = hb["좌표정보(X)"].notna() & hb["좌표정보(Y)"].notna()
eb_filter_no_location = eb["좌표정보(X)"].notna() & eb["좌표정보(Y)"].notna()

In [7]:
hb_filtered1 = hb[hb_filter_no_location]
eb_filtered1 = eb[eb_filter_no_location]
hb_filtered1.info

<bound method DataFrame.info of      개방자치단체코드                    관리번호     인허가일자  인허가취소일자  영업상태코드  영업상태명  \
0     3040000  3040000-103-1993-06863  19931030      NaN       1  영업/정상   
1     3040000  3040000-103-1993-06865  19931122      NaN       3     폐업   
2     3040000  3040000-103-1993-06871  19931026      NaN       3     폐업   
3     3040000  3040000-103-1993-06877  19931227      NaN       3     폐업   
4     3040000  3040000-103-1993-06886  19931110      NaN       3     폐업   
..        ...                     ...       ...      ...     ...    ...   
331   3040000  3040000-103-1998-07218  19981103      NaN       1  영업/정상   
332   3040000  3040000-103-1998-08630  19980216      NaN       3     폐업   
333   3040000  3040000-103-1998-08631  19980630      NaN       3     폐업   
334   3040000  3040000-103-2005-00001  20050929      NaN       3     폐업   
335   3040000  3040000-103-2012-00001  20120509      NaN       3     폐업   

     상세영업상태코드 상세영업상태명        폐업일자  휴업시작일자  ...  공장판매직종업원수  공장생산직종업원

### hb_filtered1이랑 eb_filtered1은 x,y 좌표정보가 없는 row는 삭제된 상태

### 인허가일자, 폐업일자 컬럼의 날짜에 -가 포함되어 있으면 대쉬 삭제

In [8]:
hb_filtered1 = hb_filtered1.copy()
hb_filtered1.loc[:,"인허가일자"] = hb_filtered1.loc[:,"인허가일자"].astype(str).str.replace("-","")
hb_filtered1.loc[:,"폐업일자"] = hb_filtered1.loc[:,"폐업일자"].astype(str).str.replace("-","")

In [9]:
hb_filtered1[hb_filtered1["관리번호"]=='3040000-103-1994-07011']

,개방자치단체코드,관리번호,인허가일자,인허가취소일자,영업상태코드,영업상태명,상세영업상태코드,상세영업상태명,폐업일자,휴업시작일자,...,공장판매직종업원수,공장생산직종업원수,건물소유구분명,보증액,월세액,다중이용업소여부,시설총규모,전통업소지정번호,전통업소주된음식,홈페이지
121,3040000,3040000-103-1994-07011,19940322,NaN,1,영업/정상,1,영업,nan,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
eb_filtered1 = eb_filtered1.copy()
eb_filtered1.loc[:,"인허가일자"] = eb_filtered1.loc[:,"인허가일자"].astype(str).str.replace("-","")
eb_filtered1.loc[:,"폐업일자"] = eb_filtered1.loc[:,"폐업일자"].astype(str).str.replace("-","")

# 분기별로 파일 만들기

변수명 : data연도_분기 <br>
변수명으로 df객체 생성해놓기

In [11]:
for i in range(1972,2024):
    for j in range(1,5):
        globals()[f"data{i}_{j}"] = pd.DataFrame(columns=hb_filtered1.columns)

In [12]:
print(data1972_1)

Empty DataFrame
Columns: [개방자치단체코드, 관리번호, 인허가일자, 인허가취소일자, 영업상태코드, 영업상태명, 상세영업상태코드, 상세영업상태명, 폐업일자, 휴업시작일자, 휴업종료일자, 재개업일자, 전화번호, 소재지면적, 소재지우편번호, 지번주소, 도로명주소, 도로명우편번호, 사업장명, 최종수정일자, 데이터갱신구분, 데이터갱신일자, 업태구분명, 좌표정보(X), 좌표정보(Y), 위생업태명, 남성종사자수, 여성종사자수, 영업장주변구분명, 등급구분명, 급수시설구분명, 총인원, 본사종업원수, 공장사무직종업원수, 공장판매직종업원수, 공장생산직종업원수, 건물소유구분명, 보증액, 월세액, 다중이용업소여부, 시설총규모, 전통업소지정번호, 전통업소주된음식, 홈페이지]
Index: []

[0 rows x 44 columns]


### 날짜 전달되면 data연도_분기 반환하는 함수 정의

In [13]:
ymd_date_pattern = r'(\d{4})(\d{2})(\d{2})'
def switch_quarters(string):
    string = int(string)
    if (1<=string<=3):
        return 1
    if (4<=string<=6):
        return 2
    if (7<=string<=9):
        return 3
    if (10<=string<=12):
        return 4

def sep_date(pattern,row):
    arr = []
    start_date = row["인허가일자"]
    end_date = row["폐업일자"]
    s_match = re.match(pattern,str(start_date))
    arr.append(s_match.groups()[0])
    arr.append(switch_quarters(s_match.groups()[1]))
    if (math.isnan((float)(end_date))): # 폐업을 하지 않은 업소
        arr.append(2023)
        arr.append(2)
    else:
        e_match = re.match(pattern,str(end_date))
        arr.append(e_match.groups()[0])
        arr.append(switch_quarters(e_match.groups()[1]))
    return arr

## 각 행에 대하여 해당되는 영업분기DataFrame에 append

In [14]:
for idx,row in hb_filtered1.iterrows():
    start_y,start_q,end_y,end_q = map(int,sep_date(ymd_date_pattern,row)) 
    # 각 요소를 int로 형변환
    print(idx,"번째 시작:",start_y,start_q,end_y,end_q)
    isEnd = False
    while(start_y <= end_y):
        while(start_q < 5):
            print(start_y,start_q)
            globals()[f"data{start_y}_{start_q}"].append(row,ignore_index=True)
            print("행 추가")
            if ((start_y == end_y) & (start_q == end_q)):
                isEnd = True
                print("isEnd:",isEnd)
                break
            if (start_q == 4):
                start_q=1
                break
            start_q+=1
        if isEnd:
            break
        start_y+=1
        


0 번째 시작: 1993 4 2023 2
1993 4
저장
1994 1
저장
1994 2
저장
1994 3
저장
1994 4
저장
1995 1
저장
1995 2
저장
1995 3
저장
1995 4
저장
1996 1
저장
1996 2
저장
1996 3
저장
1996 4
저장
1997 1
저장
1997 2
저장
1997 3
저장
1997 4
저장
1998 1
저장
1998 2
저장
1998 3
저장
1998 4
저장
1999 1
저장
1999 2
저장
1999 3
저장
1999 4
저장
2000 1
저장
2000 2
저장
2000 3
저장
2000 4
저장
2001 1
저장
2001 2
저장
2001 3
저장
2001 4
저장
2002 1
저장
2002 2
저장
2002 3
저장
2002 4
저장
2003 1
저장
2003 2
저장
2003 3
저장
2003 4
저장
2004 1
저장
2004 2
저장
2004 3
저장
2004 4
저장
2005 1
저장
2005 2
저장
2005 3
저장
2005 4
저장
2006 1
저장
2006 2
저장
2006 3
저장
2006 4
저장
2007 1
저장
2007 2
저장
2007 3
저장
2007 4
저장
2008 1
저장
2008 2
저장
2008 3
저장
2008 4
저장
2009 1
저장
2009 2
저장
2009 3
저장
2009 4
저장
2010 1
저장
2010 2
저장
2010 3
저장
2010 4
저장
2011 1
저장
2011 2
저장
2011 3
저장
2011 4
저장
2012 1
저장
2012 2
저장
2012 3
저장
2012 4
저장
2013 1
저장
2013 2
저장
2013 3
저장
2013 4
저장
2014 1
저장
2014 2
저장
2014 3
저장
2014 4
저장
2015 1
저장
2015 2
저장
2015 3
저장
2015 4
저장
2016 1
저장
2016 2
저장
2016 3
저장
2016 4
저장
2017 1
저장
2017 2
저장
2017 3
저장
2017 4
저장
2018 1


저장
1997 1
저장
1997 2
저장
1997 3
저장
1997 4
저장
1998 1
저장
1998 2
저장
1998 3
저장
1998 4
저장
1999 1
저장
1999 2
저장
1999 3
저장
1999 4
저장
2000 1
저장
2000 2
저장
2000 3
저장
2000 4
저장
2001 1
저장
2001 2
저장
2001 3
저장
2001 4
저장
2002 1
저장
2002 2
저장
2002 3
저장
2002 4
저장
2003 1
저장
2003 2
저장
2003 3
저장
2003 4
저장
2004 1
저장
2004 2
저장
2004 3
저장
2004 4
저장
2005 1
저장
2005 2
저장
2005 3
저장
2005 4
저장
2006 1
저장
2006 2
저장
2006 3
저장
2006 4
저장
2007 1
저장
2007 2
저장
2007 3
저장
2007 4
저장
2008 1
저장
2008 2
저장
2008 3
저장
2008 4
저장
2009 1
저장
2009 2
저장
2009 3
저장
2009 4
저장
2010 1
저장
2010 2
저장
2010 3
저장
2010 4
저장
2011 1
저장
2011 2
저장
2011 3
저장
2011 4
저장
2012 1
저장
isEnd: True
19 번째 시작: 1993 4 1999 1
1993 4
저장
1994 1
저장
1994 2
저장
1994 3
저장
1994 4
저장
1995 1
저장
1995 2
저장
1995 3
저장
1995 4
저장
1996 1
저장
1996 2
저장
1996 3
저장
1996 4
저장
1997 1
저장
1997 2
저장
1997 3
저장
1997 4
저장
1998 1
저장
1998 2
저장
1998 3
저장
1998 4
저장
1999 1
저장
isEnd: True
20 번째 시작: 1993 4 2000 4
1993 4
저장
1994 1
저장
1994 2
저장
1994 3
저장
1994 4
저장
1995 1
저장
1995 2
저장
1995 3
저장
1995 4
저장
1996 

저장
1994 1
저장
1994 2
저장
1994 3
저장
1994 4
저장
1995 1
저장
1995 2
저장
1995 3
저장
1995 4
저장
1996 1
저장
1996 2
저장
1996 3
저장
1996 4
저장
1997 1
저장
1997 2
저장
1997 3
저장
1997 4
저장
1998 1
저장
1998 2
저장
1998 3
저장
1998 4
저장
1999 1
저장
1999 2
저장
1999 3
저장
1999 4
저장
2000 1
저장
2000 2
저장
2000 3
저장
2000 4
저장
2001 1
저장
2001 2
저장
2001 3
저장
2001 4
저장
2002 1
저장
2002 2
저장
2002 3
저장
2002 4
저장
2003 1
저장
2003 2
저장
2003 3
저장
2003 4
저장
2004 1
저장
2004 2
저장
2004 3
저장
2004 4
저장
2005 1
저장
2005 2
저장
2005 3
저장
2005 4
저장
2006 1
저장
2006 2
저장
2006 3
저장
2006 4
저장
2007 1
저장
2007 2
저장
2007 3
저장
2007 4
저장
2008 1
저장
2008 2
저장
2008 3
저장
2008 4
저장
2009 1
저장
2009 2
저장
2009 3
저장
2009 4
저장
2010 1
저장
2010 2
저장
2010 3
저장
2010 4
저장
2011 1
저장
2011 2
저장
2011 3
저장
2011 4
저장
2012 1
저장
2012 2
저장
2012 3
저장
2012 4
저장
2013 1
저장
2013 2
저장
2013 3
저장
2013 4
저장
2014 1
저장
2014 2
저장
2014 3
저장
2014 4
저장
2015 1
저장
2015 2
저장
2015 3
저장
2015 4
저장
2016 1
저장
2016 2
저장
2016 3
저장
2016 4
저장
2017 1
저장
2017 2
저장
2017 3
저장
2017 4
저장
2018 1
저장
2018 2
저장
2018 3
저장
2018 4


저장
1994 3
저장
1994 4
저장
1995 1
저장
1995 2
저장
1995 3
저장
1995 4
저장
1996 1
저장
1996 2
저장
1996 3
저장
1996 4
저장
1997 1
저장
1997 2
저장
1997 3
저장
1997 4
저장
1998 1
저장
1998 2
저장
1998 3
저장
1998 4
저장
1999 1
저장
1999 2
저장
1999 3
저장
1999 4
저장
isEnd: True
55 번째 시작: 1993 4 2000 4
1993 4
저장
1994 1
저장
1994 2
저장
1994 3
저장
1994 4
저장
1995 1
저장
1995 2
저장
1995 3
저장
1995 4
저장
1996 1
저장
1996 2
저장
1996 3
저장
1996 4
저장
1997 1
저장
1997 2
저장
1997 3
저장
1997 4
저장
1998 1
저장
1998 2
저장
1998 3
저장
1998 4
저장
1999 1
저장
1999 2
저장
1999 3
저장
1999 4
저장
2000 1
저장
2000 2
저장
2000 3
저장
2000 4
저장
isEnd: True
56 번째 시작: 1993 4 2001 1
1993 4
저장
1994 1
저장
1994 2
저장
1994 3
저장
1994 4
저장
1995 1
저장
1995 2
저장
1995 3
저장
1995 4
저장
1996 1
저장
1996 2
저장
1996 3
저장
1996 4
저장
1997 1
저장
1997 2
저장
1997 3
저장
1997 4
저장
1998 1
저장
1998 2
저장
1998 3
저장
1998 4
저장
1999 1
저장
1999 2
저장
1999 3
저장
1999 4
저장
2000 1
저장
2000 2
저장
2000 3
저장
2000 4
저장
2001 1
저장
isEnd: True
57 번째 시작: 1993 4 2000 4
1993 4
저장
1994 1
저장
1994 2
저장
1994 3
저장
1994 4
저장
1995 1
저장
1995 2
저장
1995 3
저장

저장
2001 3
저장
2001 4
저장
2002 1
저장
2002 2
저장
2002 3
저장
isEnd: True
72 번째 시작: 1994 1 1997 3
1994 1
저장
1994 2
저장
1994 3
저장
1994 4
저장
1995 1
저장
1995 2
저장
1995 3
저장
1995 4
저장
1996 1
저장
1996 2
저장
1996 3
저장
1996 4
저장
1997 1
저장
1997 2
저장
1997 3
저장
isEnd: True
73 번째 시작: 1994 1 2022 1
1994 1
저장
1994 2
저장
1994 3
저장
1994 4
저장
1995 1
저장
1995 2
저장
1995 3
저장
1995 4
저장
1996 1
저장
1996 2
저장
1996 3
저장
1996 4
저장
1997 1
저장
1997 2
저장
1997 3
저장
1997 4
저장
1998 1
저장
1998 2
저장
1998 3
저장
1998 4
저장
1999 1
저장
1999 2
저장
1999 3
저장
1999 4
저장
2000 1
저장
2000 2
저장
2000 3
저장
2000 4
저장
2001 1
저장
2001 2
저장
2001 3
저장
2001 4
저장
2002 1
저장
2002 2
저장
2002 3
저장
2002 4
저장
2003 1
저장
2003 2
저장
2003 3
저장
2003 4
저장
2004 1
저장
2004 2
저장
2004 3
저장
2004 4
저장
2005 1
저장
2005 2
저장
2005 3
저장
2005 4
저장
2006 1
저장
2006 2
저장
2006 3
저장
2006 4
저장
2007 1
저장
2007 2
저장
2007 3
저장
2007 4
저장
2008 1
저장
2008 2
저장
2008 3
저장
2008 4
저장
2009 1
저장
2009 2
저장
2009 3
저장
2009 4
저장
2010 1
저장
2010 2
저장
2010 3
저장
2010 4
저장
2011 1
저장
2011 2
저장
2011 3
저장
2011 4
저장
2012 

...

저장
2000 3
저장
2000 4
저장
2001 1
저장
2001 2
저장
2001 3
저장
2001 4
저장
2002 1
저장
2002 2
저장
2002 3
저장
isEnd: True
260 번째 시작: 1996 4 2023 2
1996 4
저장
1997 1
저장
1997 2
저장
1997 3
저장
1997 4
저장
1998 1
저장
1998 2
저장
1998 3
저장
1998 4
저장
1999 1
저장
1999 2
저장
1999 3
저장
1999 4
저장
2000 1
저장
2000 2
저장
2000 3
저장
2000 4
저장
2001 1
저장
2001 2
저장
2001 3
저장
2001 4
저장
2002 1
저장
2002 2
저장
2002 3
저장
2002 4
저장
2003 1
저장
2003 2
저장
2003 3
저장
2003 4
저장
2004 1
저장
2004 2
저장
2004 3
저장
2004 4
저장
2005 1
저장
2005 2
저장
2005 3
저장
2005 4
저장
2006 1
저장
2006 2
저장
2006 3
저장
2006 4
저장
2007 1
저장
2007 2
저장
2007 3
저장
2007 4
저장
2008 1
저장
2008 2
저장
2008 3
저장
2008 4
저장
2009 1
저장
2009 2
저장
2009 3
저장
2009 4
저장
2010 1
저장
2010 2
저장
2010 3
저장
2010 4
저장
2011 1
저장
2011 2
저장
2011 3
저장
2011 4
저장
2012 1
저장
2012 2
저장
2012 3
저장
2012 4
저장
2013 1
저장
2013 2
저장
2013 3
저장
2013 4
저장
2014 1
저장
2014 2
저장
2014 3
저장
2014 4
저장
2015 1
저장
2015 2
저장
2015 3
저장
2015 4
저장
2016 1
저장
2016 2
저장
2016 3
저장
2016 4
저장
2017 1
저장
2017 2
저장
2017 3
저장
2017 4
저장
2018 1
저장
2018 2
저장


저장
1998 2
저장
1998 3
저장
1998 4
저장
1999 1
저장
1999 2
저장
1999 3
저장
1999 4
저장
2000 1
저장
2000 2
저장
2000 3
저장
2000 4
저장
2001 1
저장
2001 2
저장
2001 3
저장
2001 4
저장
2002 1
저장
2002 2
저장
2002 3
저장
2002 4
저장
2003 1
저장
2003 2
저장
2003 3
저장
2003 4
저장
2004 1
저장
2004 2
저장
2004 3
저장
2004 4
저장
2005 1
저장
2005 2
저장
2005 3
저장
2005 4
저장
2006 1
저장
2006 2
저장
2006 3
저장
2006 4
저장
2007 1
저장
2007 2
저장
2007 3
저장
2007 4
저장
2008 1
저장
2008 2
저장
2008 3
저장
2008 4
저장
2009 1
저장
2009 2
저장
2009 3
저장
2009 4
저장
2010 1
저장
2010 2
저장
2010 3
저장
2010 4
저장
2011 1
저장
2011 2
저장
2011 3
저장
2011 4
저장
2012 1
저장
2012 2
저장
2012 3
저장
2012 4
저장
2013 1
저장
2013 2
저장
2013 3
저장
2013 4
저장
2014 1
저장
2014 2
저장
2014 3
저장
2014 4
저장
2015 1
저장
2015 2
저장
2015 3
저장
2015 4
저장
2016 1
저장
2016 2
저장
2016 3
저장
2016 4
저장
2017 1
저장
2017 2
저장
2017 3
저장
2017 4
저장
2018 1
저장
2018 2
저장
2018 3
저장
2018 4
저장
2019 1
저장
2019 2
저장
2019 3
저장
2019 4
저장
2020 1
저장
2020 2
저장
2020 3
저장
2020 4
저장
2021 1
저장
2021 2
저장
2021 3
저장
2021 4
저장
2022 1
저장
isEnd: True
277 번째 시작: 1997 1 2023 2


저장
2000 3
저장
2000 4
저장
2001 1
저장
2001 2
저장
2001 3
저장
2001 4
저장
2002 1
저장
2002 2
저장
2002 3
저장
2002 4
저장
2003 1
저장
2003 2
저장
2003 3
저장
2003 4
저장
2004 1
저장
2004 2
저장
2004 3
저장
2004 4
저장
2005 1
저장
2005 2
저장
2005 3
저장
2005 4
저장
2006 1
저장
2006 2
저장
2006 3
저장
2006 4
저장
2007 1
저장
2007 2
저장
2007 3
저장
2007 4
저장
2008 1
저장
2008 2
저장
2008 3
저장
2008 4
저장
2009 1
저장
2009 2
저장
2009 3
저장
2009 4
저장
2010 1
저장
2010 2
저장
2010 3
저장
2010 4
저장
2011 1
저장
2011 2
저장
2011 3
저장
2011 4
저장
2012 1
저장
2012 2
저장
2012 3
저장
2012 4
저장
2013 1
저장
2013 2
저장
2013 3
저장
2013 4
저장
2014 1
저장
2014 2
저장
2014 3
저장
2014 4
저장
2015 1
저장
2015 2
저장
2015 3
저장
2015 4
저장
2016 1
저장
2016 2
저장
2016 3
저장
2016 4
저장
2017 1
저장
2017 2
저장
2017 3
저장
2017 4
저장
2018 1
저장
2018 2
저장
2018 3
저장
2018 4
저장
2019 1
저장
2019 2
저장
2019 3
저장
2019 4
저장
2020 1
저장
2020 2
저장
2020 3
저장
2020 4
저장
2021 1
저장
2021 2
저장
2021 3
저장
2021 4
저장
2022 1
저장
2022 2
저장
2022 3
저장
2022 4
저장
2023 1
저장
2023 2
저장
isEnd: True
292 번째 시작: 1997 2 2023 2
1997 2
저장
1997 3
저장
1997 4
저장
1998 1
저장


저장
2000 2
저장
2000 3
저장
2000 4
저장
2001 1
저장
2001 2
저장
2001 3
저장
2001 4
저장
2002 1
저장
2002 2
저장
2002 3
저장
2002 4
저장
2003 1
저장
2003 2
저장
2003 3
저장
2003 4
저장
2004 1
저장
2004 2
저장
2004 3
저장
2004 4
저장
2005 1
저장
2005 2
저장
2005 3
저장
2005 4
저장
2006 1
저장
2006 2
저장
2006 3
저장
2006 4
저장
2007 1
저장
2007 2
저장
2007 3
저장
2007 4
저장
2008 1
저장
isEnd: True
307 번째 시작: 1997 4 2023 2
1997 4
저장
1998 1
저장
1998 2
저장
1998 3
저장
1998 4
저장
1999 1
저장
1999 2
저장
1999 3
저장
1999 4
저장
2000 1
저장
2000 2
저장
2000 3
저장
2000 4
저장
2001 1
저장
2001 2
저장
2001 3
저장
2001 4
저장
2002 1
저장
2002 2
저장
2002 3
저장
2002 4
저장
2003 1
저장
2003 2
저장
2003 3
저장
2003 4
저장
2004 1
저장
2004 2
저장
2004 3
저장
2004 4
저장
2005 1
저장
2005 2
저장
2005 3
저장
2005 4
저장
2006 1
저장
2006 2
저장
2006 3
저장
2006 4
저장
2007 1
저장
2007 2
저장
2007 3
저장
2007 4
저장
2008 1
저장
2008 2
저장
2008 3
저장
2008 4
저장
2009 1
저장
2009 2
저장
2009 3
저장
2009 4
저장
2010 1
저장
2010 2
저장
2010 3
저장
2010 4
저장
2011 1
저장
2011 2
저장
2011 3
저장
2011 4
저장
2012 1
저장
2012 2
저장
2012 3
저장
2012 4
저장
2013 1
저장
2013 2
저장
2013 3
저장


저장
2021 2
저장
2021 3
저장
2021 4
저장
2022 1
저장
2022 2
저장
2022 3
저장
2022 4
저장
2023 1
저장
2023 2
저장
isEnd: True
321 번째 시작: 1998 1 2001 1
1998 1
저장
1998 2
저장
1998 3
저장
1998 4
저장
1999 1
저장
1999 2
저장
1999 3
저장
1999 4
저장
2000 1
저장
2000 2
저장
2000 3
저장
2000 4
저장
2001 1
저장
isEnd: True
322 번째 시작: 1998 1 2001 3
1998 1
저장
1998 2
저장
1998 3
저장
1998 4
저장
1999 1
저장
1999 2
저장
1999 3
저장
1999 4
저장
2000 1
저장
2000 2
저장
2000 3
저장
2000 4
저장
2001 1
저장
2001 2
저장
2001 3
저장
isEnd: True
324 번째 시작: 1998 1 2012 4
1998 1
저장
1998 2
저장
1998 3
저장
1998 4
저장
1999 1
저장
1999 2
저장
1999 3
저장
1999 4
저장
2000 1
저장
2000 2
저장
2000 3
저장
2000 4
저장
2001 1
저장
2001 2
저장
2001 3
저장
2001 4
저장
2002 1
저장
2002 2
저장
2002 3
저장
2002 4
저장
2003 1
저장
2003 2
저장
2003 3
저장
2003 4
저장
2004 1
저장
2004 2
저장
2004 3
저장
2004 4
저장
2005 1
저장
2005 2
저장
2005 3
저장
2005 4
저장
2006 1
저장
2006 2
저장
2006 3
저장
2006 4
저장
2007 1
저장
2007 2
저장
2007 3
저장
2007 4
저장
2008 1
저장
2008 2
저장
2008 3
저장
2008 4
저장
2009 1
저장
2009 2
저장
2009 3
저장
2009 4
저장
2010 1
저장
2010 2
저장
2010 3
저장
2010 4

In [16]:
data1988_1

,개방자치단체코드,관리번호,인허가일자,인허가취소일자,영업상태코드,영업상태명,상세영업상태코드,상세영업상태명,폐업일자,휴업시작일자,...,공장판매직종업원수,공장생산직종업원수,건물소유구분명,보증액,월세액,다중이용업소여부,시설총규모,전통업소지정번호,전통업소주된음식,홈페이지


In [15]:
for i in range(1972,2024):
    for j in range(1,5):
        globals()[f"data{i}_{j}"].to_csv(f"./data/bars/data{i}_{j}.csv",index=False)